# Project: Data Cleaning - Tidy up messy Datasets (Movies Dataset)

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 20)
pd.set_option('display.max_columns', None)
import numpy as np
import ast

class MovieData:
    def __init__(self, file_name):
        self._raw_data = pd.read_csv(file_name, 
                                     low_memory= False, 
                                     na_values= '[]')
        self._clean_data = self._data_preprocessing(self._raw_data)
        
        
        
    def _data_preprocessing(self, data):
        df = data.copy()
        
        dropped_cols = ['adult', 'imdb_id', 'original_title', 'video', 'homepage']
        df.drop(columns = dropped_cols, 
                inplace = True)
        
        nested_cols = ["belongs_to_collection", "genres", 
                       "production_countries", "production_companies", 
                       "spoken_languages"]
        for col in nested_cols:
            df[col] = df[col].apply(self.JSON_Cleaner, attr=  'name')
            
            
        df['overview'] = df['overview'].apply(self.overview_cleaner)
        
        numeric_cols = ["budget", "id", "popularity", 'revenue']
        for col in numeric_cols:
            df[col] = pd.to_numeric(df[col], errors= 'coerce')
            
        
        df['budget'] = round(df['budget'] / 1000000, 2)
        df['revenue'] = round(df['revenue'] / 1000000, 2)
        df['popularity'] = round(df['popularity'], 2)
        
        df['release_date'] = pd.to_datetime(df['release_date'], 
                                            format = '%Y-%m-%d', 
                                            errors = 'coerce')
        
        unreasonable_info_idx = df.loc[(
                ((df["budget"] == 0)|(df["revenue"] == 0)) &
                (df["runtime"] == 0)
                )].index
        df.drop(index = unreasonable_info_idx, inplace = True)
        
        
        
        
        
        return df.reset_index(drop = True).drop_duplicates()
    
    
    @staticmethod
    def JSON_Cleaner(element, attr):
        if isinstance(element, str):
            data = ast.literal_eval(element)
            if isinstance(data, dict):
                return data[attr]
            elif isinstance(data, list):
                if len(data) == 1:
                    return data[0][attr]
                elif len(data) > 1:
                    return '|'.join(i[attr] for i in data)
        else:
            return element
    @staticmethod
    def overview_cleaner(element):
        null_overview = ['No overview found.', 
                 'No Overview', ' ', 
                 'No movie overview available.']
        if element in null_overview:
            return np.nan
        else:
            return element
        

In [2]:
movie = MovieData(file_name= 'movies_metadata.csv')

# Original Raw Data
- The original DataFrame is unstructured and unclean, which contasin __nested JSON-structued Data__, with other meaningless information as shown in <code> movie._raw_data</code>

In [3]:
movie._raw_data.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'n...",30000000,"[{'id': 16, 'nam...",http://toystory....,862,tt0114709,en,Toy Story,"Led by Woody, An...",21.946943,/rhIRbceoE9lR4ve...,[{'name': 'Pixar...,[{'iso_3166_1': ...,1995-10-30,373554033.0,81.0,[{'iso_639_1': '...,Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'nam...",NaN,8844,tt0113497,en,Jumanji,When siblings Ju...,17.015539,/vzmL6fP7aPKNKPR...,[{'name': 'TriSt...,[{'iso_3166_1': ...,1995-12-15,262797249.0,104.0,[{'iso_639_1': '...,Released,Roll the dice an...,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, '...",0,"[{'id': 10749, '...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding...,11.7129,/6ksm1sjKMFLbO7U...,[{'name': 'Warne...,[{'iso_3166_1': ...,1995-12-22,0.0,101.0,[{'iso_639_1': '...,Released,Still Yelling. S...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mist...",3.859495,/16XOMpEaLWkrcPq...,[{'name': 'Twent...,[{'iso_3166_1': ...,1995-12-22,81452156.0,127.0,[{'iso_639_1': '...,Released,Friends are the ...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'n...",0,"[{'id': 35, 'nam...",NaN,11862,tt0113041,en,Father of the Br...,Just when George...,8.387519,/e64sOI48hQXyru7...,[{'name': 'Sando...,[{'iso_3166_1': ...,1995-02-10,76578911.0,106.0,[{'iso_639_1': '...,Released,Just When His Wo...,Father of the Br...,False,5.7,173.0


In [4]:
nested_cols = ["belongs_to_collection", "genres", 
                       "production_countries", "production_companies", 
                       "spoken_languages"]

movie_nested = movie._raw_data[nested_cols].copy()
movie_nested.head(5)

,belongs_to_collection,genres,production_countries,production_companies,spoken_languages
0,"{'id': 10194, 'n...","[{'id': 16, 'nam...",[{'iso_3166_1': ...,[{'name': 'Pixar...,[{'iso_639_1': '...
1,NaN,"[{'id': 12, 'nam...",[{'iso_3166_1': ...,[{'name': 'TriSt...,[{'iso_639_1': '...
2,"{'id': 119050, '...","[{'id': 10749, '...",[{'iso_3166_1': ...,[{'name': 'Warne...,[{'iso_639_1': '...
3,NaN,"[{'id': 35, 'nam...",[{'iso_3166_1': ...,[{'name': 'Twent...,[{'iso_639_1': '...
4,"{'id': 96871, 'n...","[{'id': 35, 'nam...",[{'iso_3166_1': ...,[{'name': 'Sando...,[{'iso_639_1': '...


# Data Cleaning
- Cleaning up the data that contains __nested-JSON data__ with static method <code>JSON_Cleaner</code>. The resulting __cleaned DataFrame__ is shown below

### How to flatten the nested Columns
1. __Extract__ only the __collection name__ from the column "belongs_to_collection" and __overwrite__ "belongs_to_collection". <br> For example: The value in the first row (Toy Story) should be 'Toy Story Collection'.

2. __Extract__ all __genre names__ from the column "genres" and __overwrite__ "genres". If a movie has more than one genre, __seperate genres by a pipe__ "|".<br>
For example: The value in the first row (Toy Story) should be 'Animation|Comedy|Family'.

3. __Extract__ all __spoken language names__ from the column "spoken_languages" and __overwrite__ "spoken_languages". If a movie has more than one spoken language, __seperate spoken languages by a pipe__ "|".<br>
For example: The value in the first row (Toy Story) should be 'English'.

4. __Extract__ all __production countries names__ from the column "production_countries" and __overwrite__ "production_countries". If a movie has more than one production country, __seperate production countries by a pipe__ "|".<br>
For example: The value in the first row (Toy Story) should be 'United States of America'.

5. __Extract__ all __production companies names__ from the column "production_companies" and __overwrite__ "production_companies". If a movie has more than one production company, __seperate production companies by a pipe__ "|".<br>
For example: The value in the first row (Toy Story) should be 'Pixar Animation Studios'

In [5]:
for col in movie_nested.columns:
    movie_nested[col] = movie_nested[col].apply(movie.JSON_Cleaner, attr = 'name')

In [6]:
movie_nested.head()

,belongs_to_collection,genres,production_countries,production_companies,spoken_languages
0,Toy Story Collec...,Animation|Comedy...,United States of...,Pixar Animation ...,English
1,NaN,Adventure|Fantas...,United States of...,TriStar Pictures...,English|Français
2,Grumpy Old Men C...,Romance|Comedy,United States of...,Warner Bros.|Lan...,English
3,NaN,Comedy|Drama|Rom...,United States of...,Twentieth Centur...,English
4,Father of the Br...,Comedy,United States of...,Sandollar Produc...,English


### Column __Overview__  contains meaningless inforamtion !!
- It is obvious that column __overview__ contains meaningless inforamtion (e.g., 'No overview', 'No overview available', 'No overview found' etc.), so I decide to clean it up with another static method <code>overview_cleaner</code>

In [7]:
movie._raw_data['overview'].value_counts(dropna = False).nlargest(6)

NaN                                     954
No overview found.                      133
No Overview                               7
                                          5
No movie overview available.              3
Adaptation of the Jane Austen novel.      3
Name: overview, dtype: int64

In [8]:
movie._raw_data['overview'].apply(movie.overview_cleaner).value_counts(dropna = False).nlargest(1)

NaN    1102
Name: overview, dtype: int64

##### After above operation, column __overview__ is cleaned up the cell that contains __inforamtion listed below__ are all categorized as missing values (Go from 954 to 1102)
1. 'No overview', 
2. 'No overview available', 
3. 'No overview found', 
4. '' 

## Numerical Columns
1. __Convert__ the datatype in the columns __"budget"__, __"id"__, __"revenue"__, and __"popularity"__ __to numeric__. Set invalid values as NaN.
2. __Convert__ columns "budget" and "revenue" into Million USD and round it up to 2 decimal points.

In [9]:
numeric_cols = ["budget", "id", "popularity", 'revenue']

numeric_df = movie._raw_data[numeric_cols].copy()

In [10]:
numeric_df.dtypes

budget         object
id             object
popularity     object
revenue       float64
dtype: object

In [11]:
for col in numeric_cols:
    numeric_df[col] = pd.to_numeric(numeric_df[col], errors= 'coerce')

In [12]:
numeric_df.dtypes

budget        float64
id            float64
popularity    float64
revenue       float64
dtype: object

In [13]:
numeric_df['budget'] = round(numeric_df['budget'] / 1000000, 2)
numeric_df['revenue'] = round(numeric_df['revenue'] / 1000000, 2)
numeric_df['popularity'] = round(numeric_df['popularity'], 2)

In [14]:
numeric_df

,budget,id,popularity,revenue
0,30.0,862.0,21.95,373.55
1,65.0,8844.0,17.02,262.80
2,0.0,15602.0,11.71,0.00
3,16.0,31357.0,3.86,81.45
4,0.0,11862.0,8.39,76.58
...,...,...,...,...
45461,0.0,439050.0,0.07,0.00
45462,0.0,111109.0,0.18,0.00
45463,0.0,67758.0,0.90,0.00
45464,0.0,227506.0,0.00,0.00


## Datetime Column
 - __Convert__ the object data type column __'release_date'__ into datetime data type

In [15]:
movie._raw_data['release_date'] = pd.to_datetime(movie._raw_data['release_date'], 
               format ='%Y-%m-%d', 
               errors = 'coerce')

In [16]:
movie._raw_data['release_date']

0       1995-10-30
1       1995-12-15
2       1995-12-22
3       1995-12-22
4       1995-02-10
           ...    
45461          NaT
45462   2011-11-17
45463   2003-08-01
45464   1917-10-21
45465   2017-06-09
Name: release_date, Length: 45466, dtype: datetime64[ns]

#### Drop the movie information that is unreasonable (i.e., runtime, 'revenue', 'budget' all being 0)

In [17]:
movie._raw_data.loc[(
    ((movie._raw_data["budget"] == 0)|(movie._raw_data["revenue"] == 0)) &
    (movie._raw_data["runtime"] == 0))
                ,['title',"budget", "revenue" ,"runtime"]]

,title,budget,revenue,runtime
222,Dream Man,0,0.0,0.0
224,Destiny Turns on...,0,0.0,0.0
398,Dos Crímenes,0,0.0,0.0
554,The Beans of Egy...,0,0.0,0.0
667,The Run of the C...,0,0.0,0.0
...,...,...,...,...
45360,Up 'n' Under,0,0.0,0.0
45370,How Most Things ...,0,0.0,0.0
45371,LEGO DC Super He...,0,0.0,0.0
45416,"Whiffles, Cubic ...",0,0.0,0.0


In [18]:
movie._raw_data.drop(index = movie._raw_data.loc[(
    ((movie._raw_data["budget"] == 0)|(movie._raw_data["revenue"] == 0)) &
    (movie._raw_data["runtime"] == 0))
                ,['title',"budget", "revenue" ,"runtime"]].index
                    ).head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'n...",30000000,"[{'id': 16, 'nam...",http://toystory....,862,tt0114709,en,Toy Story,"Led by Woody, An...",21.946943,/rhIRbceoE9lR4ve...,[{'name': 'Pixar...,[{'iso_3166_1': ...,1995-10-30,373554033.0,81.0,[{'iso_639_1': '...,Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'nam...",NaN,8844,tt0113497,en,Jumanji,When siblings Ju...,17.015539,/vzmL6fP7aPKNKPR...,[{'name': 'TriSt...,[{'iso_3166_1': ...,1995-12-15,262797249.0,104.0,[{'iso_639_1': '...,Released,Roll the dice an...,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, '...",0,"[{'id': 10749, '...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding...,11.7129,/6ksm1sjKMFLbO7U...,[{'name': 'Warne...,[{'iso_3166_1': ...,1995-12-22,0.0,101.0,[{'iso_639_1': '...,Released,Still Yelling. S...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mist...",3.859495,/16XOMpEaLWkrcPq...,[{'name': 'Twent...,[{'iso_3166_1': ...,1995-12-22,81452156.0,127.0,[{'iso_639_1': '...,Released,Friends are the ...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'n...",0,"[{'id': 35, 'nam...",NaN,11862,tt0113041,en,Father of the Br...,Just when George...,8.387519,/e64sOI48hQXyru7...,[{'name': 'Sando...,[{'iso_3166_1': ...,1995-02-10,76578911.0,106.0,[{'iso_639_1': '...,Released,Just When His Wo...,Father of the Br...,False,5.7,173.0


# Cleaned Data Presentation

In [19]:
movie._clean_data

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,Toy Story Collec...,30.0,Animation|Comedy...,862.0,en,"Led by Woody, An...",21.95,/rhIRbceoE9lR4ve...,Pixar Animation ...,United States of...,1995-10-30,373.55,81.0,English,Released,NaN,Toy Story,7.7,5415.0
1,NaN,65.0,Adventure|Fantas...,8844.0,en,When siblings Ju...,17.02,/vzmL6fP7aPKNKPR...,TriStar Pictures...,United States of...,1995-12-15,262.80,104.0,English|Français,Released,Roll the dice an...,Jumanji,6.9,2413.0
2,Grumpy Old Men C...,0.0,Romance|Comedy,15602.0,en,A family wedding...,11.71,/6ksm1sjKMFLbO7U...,Warner Bros.|Lan...,United States of...,1995-12-22,0.00,101.0,English,Released,Still Yelling. S...,Grumpier Old Men,6.5,92.0
3,NaN,16.0,Comedy|Drama|Rom...,31357.0,en,"Cheated on, mist...",3.86,/16XOMpEaLWkrcPq...,Twentieth Centur...,United States of...,1995-12-22,81.45,127.0,English,Released,Friends are the ...,Waiting to Exhale,6.1,34.0
4,Father of the Br...,0.0,Comedy,11862.0,en,Just when George...,8.39,/e64sOI48hQXyru7...,Sandollar Produc...,United States of...,1995-02-10,76.58,106.0,English,Released,Just When His Wo...,Father of the Br...,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43913,NaN,0.0,Drama|Family,439050.0,fa,Rising and falli...,0.07,/jldsYflnId4tTWP...,NaN,Iran,NaT,0.00,90.0,فارسی,Released,Rising and falli...,Subdue,4.0,1.0
43914,NaN,0.0,Drama,111109.0,tl,An artist strugg...,0.18,/xZkmxsNmYXJbKVs...,Sine Olivia,Philippines,2011-11-17,0.00,360.0,,Released,NaN,Century of Birthing,9.0,3.0
43915,NaN,0.0,Action|Drama|Thr...,67758.0,en,When one of her ...,0.90,/d5bX92nDsISNhu3...,American World P...,United States of...,2003-08-01,0.00,90.0,English,Released,A deadly game of...,Betrayal,3.8,6.0
43916,NaN,0.0,NaN,227506.0,en,In a small town ...,0.00,/aorBPO7ak8e8iJK...,Yermoliev,Russia,1917-10-21,0.00,87.0,NaN,Released,NaN,Satan Triumphant,0.0,0.0
